In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

def preprocess_data(caminho_csv):
    df = pd.read_csv(caminho_csv, delimiter=',')

    df.drop(columns=['ID', 'Argila', 'Areia', 'Silte', 'Torque Max. (kgf.m)',
                     'E(Mpa)', 'Nspt_qc_product'], inplace=True, errors='ignore')

    df['Nspt'] = df['Nspt'].interpolate()

    df = df.assign(
        Nspt_qc_ratio=lambda x: x['Nspt'] / (x['qc(Mpa)'] + 1e-8),
        Depth_fs_product=lambda x: x['Deph(m)'] * x['fs(Mpa)'],
        Nspt_carga_product=lambda x: x['Nspt'] * x['Carga_Traçao'],
        log_Nspt=lambda x: np.log1p(x['Nspt'])
    )

    Q1 = df['Nspt_carga_product'].quantile(0.25)
    Q3 = df['Nspt_carga_product'].quantile(0.75)
    IQR = Q3 - Q1
    lim_inf = Q1 - 1.5 * IQR
    lim_sup = Q3 + 1.5 * IQR
    df['Nspt_carga_product'] = np.clip(df['Nspt_carga_product'], lim_inf, lim_sup)
    df['log_Nspt_carga_product'] = np.log1p(df['Nspt_carga_product'])

    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    numeric_cols = df.select_dtypes(include=np.number).columns
    imputer = IterativeImputer(random_state=42)
    df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

    cols_sem_negativos = ['Deph(m)', 'Nspt', 'qc(Mpa)', 'fs(Mpa)', 'Depth_fs_product', 'Nspt_qc_ratio']
    df[cols_sem_negativos] = df[cols_sem_negativos].clip(lower=0)

    scaler = MinMaxScaler()
    cols_norm = cols_sem_negativos + ['log_Nspt_carga_product']
    df[cols_norm] = scaler.fit_transform(df[cols_norm])

    scaler_carga = MinMaxScaler()
    df['Carga_Traçao'] = scaler_carga.fit_transform(df['Carga_Traçao'].values.reshape(-1, 1)).flatten()

    X = df.drop(columns=['Carga_Traçao'])
    y = df['Carga_Traçao']

    return X, y, scaler_carga, scaler

def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    xgb = XGBRegressor(n_estimators=100, random_state=42, verbosity=0)

    rf.fit(X_train, y_train)
    xgb.fit(X_train, y_train)

    rf_pred = rf.predict(X_test)
    xgb_pred = xgb.predict(X_test)

    ensemble_pred = (rf_pred + xgb_pred) / 2

    mse = mean_squared_error(y_test, ensemble_pred)
    r2 = r2_score(y_test, ensemble_pred)

    print(f"\n📊 Avaliação do Ensemble:")
    print(f"MSE: {mse:.4f}")
    print(f"R²:  {r2:.4f}")

    return rf, xgb

def predict_new_case(rf, xgb, scaler, scaler_carga):
    novo = pd.DataFrame({
        'Deph(m)': [15],
        'Nspt': [25],
        'qc(Mpa)': [12.5],
        'fs(Mpa)': [0.15]
    })
    novo['Nspt_qc_ratio'] = novo['Nspt'] / (novo['qc(Mpa)'] + 1e-8)
    novo['Depth_fs_product'] = novo['Deph(m)'] * novo['fs(Mpa)']
    novo['Nspt_carga_product'] = novo['Nspt'] * 1  # valor fictício
    novo['log_Nspt'] = np.log1p(novo['Nspt'])
    novo['log_Nspt_carga_product'] = np.log1p(novo['Nspt_carga_product'])

    cols_ordem = ['Deph(m)', 'Nspt', 'qc(Mpa)', 'fs(Mpa)', 'Nspt_qc_ratio',
                  'Depth_fs_product', 'Nspt_carga_product', 'log_Nspt', 'log_Nspt_carga_product']

    novo[cols_ordem[:-2] + ['log_Nspt_carga_product']] = scaler.transform(
        novo[cols_ordem[:-2] + ['log_Nspt_carga_product']]
    )

    rf_pred = rf.predict(novo)
    xgb_pred = xgb.predict(novo)
    ensemble_pred = (rf_pred + xgb_pred) / 2

    pred_real = scaler_carga.inverse_transform(ensemble_pred.reshape(-1, 1))

    print(f"\n🔮 Previsão para nova estaca:")
    print(f"Capacidade de Carga prevista: {pred_real[0][0]:.2f} kN")

def main():
    print("🚀 Iniciando Pipeline Ensemble...")
    X, y, scaler_carga, scaler = preprocess_data('base_dataset3_carga.csv')
    rf, xgb = train_model(X, y)
    predict_new_case(rf, xgb, scaler, scaler_carga)

if __name__ == "__main__":
    main()
